In [1]:
try:
    from copulas.multivariate import GaussianMultivariate
except:
    %pip install copulas
    from copulas.multivariate import GaussianMultivariate

try:
    import autograd.numpy as np  # Thinly-wrapped numpy
    from autograd import grad    # The only autograd function you may ever need
    from autograd.scipy.stats import norm, chi2
except:
    %pip install autograd
    import autograd.numpy as np  # Thinly-wrapped numpy
    from autograd import grad    # The only autograd function you may ever need
    from autograd.scipy.stats import norm, chi2

import pandas as pd
import matplotlib.pyplot as plt
from turtle import color
import math
from sklearn.preprocessing import QuantileTransformer
from scipy.optimize import minimize
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
def create_synthetic(X, y):
    """
    This function combines X and y into a single dataset D, models it
    using a Gaussian copula, and generates a synthetic dataset S. It
    returns the new, synthetic versions of X and y.
    """
    dataset = np.concatenate([X, np.expand_dims(y, 1)], axis=1)

    model = GaussianMultivariate()
    model.fit(dataset)

    synthetic = model.sample(len(dataset))

    X = synthetic.values[:, :-1]
    y = synthetic.values[:, -1]

    return X, y


In [6]:
BIDS = pd.read_csv('Data_for_kdensity.csv')
BIDS

,roy_dm,lncpad_dm,accbid
0,0.012741,-0.142119,1
1,-0.020593,-0.036445,0
2,0.008937,0.055362,1
3,-0.024396,0.098425,0
4,-0.001684,1.158188,1
...,...,...,...
899,-0.051214,-0.321059,0
900,0.022731,-0.501388,0
901,0.022731,-0.180459,1
902,-0.031481,0.437533,1


In [13]:
BIDS_WIN = BIDS[BIDS['accbid'] == 1]


pd.concat([BIDS[BIDS['accbid'] == 1], BIDS[BIDS['accbid'] == 0]], axis=1, ignore_index=True)

,0,1,2,3,4,5
0,0.012741,-0.142119,1.0,NaN,NaN,NaN
2,0.008937,0.055362,1.0,NaN,NaN,NaN
4,-0.001684,1.158188,1.0,NaN,NaN,NaN
6,-0.000209,1.693100,1.0,NaN,NaN,NaN
8,-0.026788,0.605619,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...
894,NaN,NaN,NaN,0.042153,0.649905,0.0
896,NaN,NaN,NaN,0.006287,-0.792542,0.0
899,NaN,NaN,NaN,-0.051214,-0.321059,0.0
900,NaN,NaN,NaN,0.022731,-0.501388,0.0


In [27]:
BIDS_WIN   = BIDS[BIDS['accbid'] == 1].reset_index().drop(columns=['index', 'accbid']).rename(columns={'roy_dm' : 'a', 'lncpad_dm' : 'b'})
BIDS_LOSS  = BIDS[BIDS['accbid'] == 0].reset_index().drop(columns=['index', 'accbid']).rename(columns={'roy_dm' : 'a0', 'lncpad_dm' : 'b0'})
SAMPLE = pd.concat([BIDS_WIN, BIDS_LOSS], axis=1)

In [28]:
model = GaussianMultivariate()
SYNTHETIC = model.fit(SAMPLE)

SYNTHETIC = model.sample(5000)

SYNTHETIC

,a,b,a0,b0
0,-0.024929,0.801569,-0.041859,-0.855296
1,0.024391,1.027610,0.063741,0.378114
2,-0.011887,0.141055,0.032910,-0.713738
3,-0.037334,0.318827,-0.048890,-0.503672
4,0.007490,0.149895,-0.017522,-1.506368
...,...,...,...,...
4995,0.034468,0.943598,0.053048,0.751719
4996,-0.014294,1.352490,-0.028066,0.427527
4997,0.028307,1.410374,0.040087,1.186189
4998,0.006977,0.085544,0.007089,-0.229014
